In [1]:
import pandas as pd

YEAR = 2024
url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
data = pd.read_csv(url, compression='gzip', low_memory=False)

playoff_data = data[data['week'].isin([19, 20, 21])]

if all(col in playoff_data.columns for col in ['passer_player_name', 'passing_yards', 'yards_after_catch']):
    passing_stats = playoff_data.groupby('passer_player_name', as_index=False).agg({
        'passing_yards': 'sum',
        'yards_after_catch': 'sum'
    })


    passing_stats['yac_percentage'] = (passing_stats['yards_after_catch'] / passing_stats['passing_yards']) * 100

    passing_stats = passing_stats.sort_values(by='passing_yards', ascending=False)
    print(passing_stats)
else:
    print("The required columns ('passer_player_name', 'passing_yards', 'yards_after_catch') are not present in the data.")


   passer_player_name  passing_yards  yards_after_catch  yac_percentage
5           J.Daniels          822.0              432.0       52.554745
4             J.Allen          636.0              322.0       50.628931
13         M.Stafford          533.0              179.0       33.583490
2            C.Stroud          527.0              180.0       34.155598
9             J.Hurts          505.0              303.0       60.000000
12          L.Jackson          429.0              171.0       39.860140
14          P.Mahomes          422.0              328.0       77.725118
7              J.Goff          313.0              153.0       48.881789
16           R.Wilson          270.0              126.0       46.666667
17          S.Darnold          245.0               99.0       40.408163
8           J.Herbert          242.0              112.0       46.280992
10             J.Love          212.0              127.0       59.905660
0          B.Mayfield          185.0               81.0       43

In [36]:
import pandas as pd
import plotly.graph_objects as go

background_color = "#15202B"
gray_color = "#9f9f9f"
highlight_color = "#01C4E7"

players_to_include = [
    'J.Daniels', 'J.Allen', 'M.Stafford', 'L.Jackson', 'P.Mahomes',
    'J.Goff', 'R.Wilson', 'S.Darnold', 'J.Hurts', 'J.Herbert',
    'J.Love', 'B.Mayfield', 'B.Nix'
]

filtered_stats = passing_stats[passing_stats['passer_player_name'].isin(players_to_include)]

filtered_stats = filtered_stats.sort_values(by='yac_percentage', ascending=True).reset_index(drop=True)

total_yards = filtered_stats['passing_yards']  
yac_yards = filtered_stats['yards_after_catch'] 

fig = go.Figure()

fig.add_trace(go.Bar(
    y=filtered_stats['passer_player_name'],
    x=total_yards - yac_yards,
    name='Air Yards',
    orientation='h',
    marker_color=gray_color,
    texttemplate=None,
    showlegend=True
))

fig.add_trace(go.Bar(
    y=filtered_stats['passer_player_name'],
    x=yac_yards,
    name='YAC',
    orientation='h',
    marker_color=highlight_color,
    texttemplate=None,
    showlegend=True
))

for i, row in filtered_stats.iterrows():
    fig.add_annotation(
        y=row['passer_player_name'],
        x=row['yards_after_catch'] + (row['passing_yards'] - row['yards_after_catch']),
        text=f"{row['yac_percentage']:.1f}%",
        showarrow=False,
        font=dict(size=12, color="white"),
        xanchor="left"
    )

fig.add_layout_image(
    dict(
        source="1.png",   
        xref="paper",
        yref="paper",
        x=1,               
        y=0,               
        xanchor="right",
        yanchor="bottom",
        sizex=0.15,        
        sizey=0.15,
        sizing="contain",
        opacity=1,
        layer="above"
    )
)

fig.update_layout(
    title=dict(
        text="2024 Playoff Passing Yards Leaders<br><span style='font-size:18px;color:lightgray'>Ordered by YAC Contribution to Total</span>",
        x=0.5,
        font=dict(size=20, color="white")
    ),
    barmode='stack',
    xaxis=dict(
        title="",
        title_font=dict(size=16, color="white"),
        tickfont=dict(size=14, color="white"),
        showgrid=False
    ),
    yaxis=dict(
        title=None,
        tickfont=dict(size=14, color="white")
    ),
    legend=dict(
        orientation="v",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(size=12, color="white")
    ),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    margin=dict(l=100, r=20, t=80, b=60),

    height=800, 
    width=600   
)

fig.add_annotation(
    text="Ray Carpenter | @array_carpenter | TheSpade.Substack.com | Data: NFLVerse",
    xref="paper",
    yref="paper",
    x=0.5,
    y=-.05,    
    xanchor="center",
    yanchor="top",
    showarrow=False,
    font=dict(size=12, color="white")
)

fig.show()
